In [1]:
import openai
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)


In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "save_food_log",
            "description": "Записать продукт в дневник питания",
            "parameters": {
                "type": "object",
                "properties": {
                    "food_item": {"type": "string", "description": "Название продукта"},
                    "calories": {"type": "integer", "description": "Калории"},
                    "protein": {"type": "integer"},
                    'count': {'type':['string', 'integer'], 'description':'Количество порций или вес продукта'},
                },
                "required": ["food_item", "calories", 'protein', 'count'],
            }
        }
    }
]

In [4]:
def ask_agent(user_query, client):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Ты эксперт-нутрициолог. Для записи еды ВСЕГДА используй функцию save_food_log."},
            {"role": "user", "content": user_query}
        ],
        tools=tools,
        tool_choice="auto"
    )
    return response.choices[0].message

In [5]:
user_query = 'Я съел два сваренных яйца и яблоко, а также я съел омлет был на 200 калорий и 14 грамм белка, а яблоко на 95 калорий и 0.5 грамм белка. Запиши это в мой дневник питания.'
answer = ask_agent(user_query, client)

In [6]:
import pandas as pd

saved_food = pd.DataFrame(columns=['food_item', 'calories', 'protein', 'count'])

In [7]:
def save_food_log(food_item: str, calories: int, protein: int, count, df):
    return pd.concat([df, pd.DataFrame([{
                'food_item': food_item,
                'calories': calories, 
                'protein': protein, 
                'count': count}])], ignore_index=True)

In [8]:
import json

if answer.tool_calls:
    for tool_call in answer.tool_calls:
        if tool_call.function.name == "save_food_log":

            arguments_raw = tool_call.function.arguments
            arguments = json.loads(arguments_raw)

            saved_food = save_food_log(
                food_item=arguments['food_item'],
                calories=arguments['calories'],
                protein=arguments['protein'],
                count=arguments['count'],
                df=saved_food
            )

In [9]:
saved_food

,food_item,calories,protein,count
0,омлет,200,14,1
1,яблоко,95,0.5,1
2,яйцо сваренное,155,13,2


In [11]:
import pandas as pd
from agent import FoodTrackerAgent


agent = FoodTrackerAgent()
df = pd.DataFrame()
df = agent.save_food_log(answer, df)

In [12]:
df

,food_item,calories,protein,count
0,омлет,200,14.0,1
1,яблоко,95,0.5,1
2,яйцо сваренное,155,13.0,2
